In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import scipy.optimize as optimization

from BHDVCStf import BHDVCS #modified bhdvcs file
import utilities as uts

In [7]:
bhdvcs = BHDVCStf.BHDVCS()

In [11]:
df = pd.read_csv("dvcs_psuedo.csv")
data = uts.DvcsData(df)

numSets = 15
numReplicas = 500

In [12]:
df

,#Set,index,errF,F,phi_x,QQ,x_b,t,k,F1,F2,dvcs,ReH,ReE,ReHtilde
0,0,0,0.002910,0.058205,0,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
1,0,1,0.002780,0.055596,10,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
2,0,2,0.003318,0.066353,20,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
3,0,3,0.003433,0.068655,30,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
4,0,4,0.003638,0.072765,40,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,13.0554,-53.0554,7.25302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,14,31,0.001199,0.023981,310,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
536,14,32,0.001267,0.025345,320,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
537,14,33,0.001210,0.024191,330,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283
538,14,34,0.001027,0.020530,340,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,11.7411,-51.7411,6.52283


In [390]:
def random_poly(df, max_terms=10, max_coef=10):
    numKins = np.random.randint(1, 5)
    kins = list(itertools.combinations(['x_b', 'QQ', 't', 'k'], numKins))
    kins = kins[np.random.randint(len(kins))]
    tot = np.zeros(len(df))
    for kin in kins:
        numTerms = np.random.randint(1, max_terms+1)
        exps = np.random.choice(range(max_terms), size=numTerms, replace=False)
        coefs = np.random.choice(range(-max_coef, max_coef+1), size=numTerms, replace=False)
        for c, e in zip(coefs, exps):
            tot += c*df[kin]**e
    return tot

In [391]:
def gen_pseudo(df, error=.05):
    dfdata = uts.DvcsData(df)

    tc = df.copy()
    tc['ReH'] = random_poly(tc)
    tc['ReE'] = random_poly(tc)
    tc['ReHtilde'] = random_poly(tc)
    #raise Error
    tc['F'] = bhdvcs.TotalUUXS(np.array(dfdata.XnoCFF), tc['ReH'], tc['ReE'], tc['ReHtilde'])
    tc['errF'] = tc['F']*error
    return tc

In [404]:
test = gen_pseudo(df)

In [405]:
test

,#Set,index,errF,F,phi_x,QQ,x_b,t,k,F1,F2,dvcs,ReH,ReE,ReHtilde
0,0,0,2685.886186,5.371772e+04,0,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,-9.686440e+05,-5331.665039,1.026722e+06
1,0,1,2539.615939,5.079232e+04,10,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,-9.686440e+05,-5331.665039,1.026722e+06
2,0,2,2160.192741,4.320385e+04,20,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,-9.686440e+05,-5331.665039,1.026722e+06
3,0,3,1677.072679,3.354145e+04,30,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,-9.686440e+05,-5331.665039,1.026722e+06
4,0,4,1205.387183,2.410774e+04,40,1.74013,0.435095,-0.380868,3.75,0.498060,0.68579,0.012288,-9.686440e+05,-5331.665039,1.026722e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,14,31,126928.150786,2.538563e+06,310,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,-6.187640e+08,-196307.711914,7.060131e+08
536,14,32,189444.380994,3.788888e+06,320,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,-6.187640e+08,-196307.711914,7.060131e+08
537,14,33,261949.839564,5.238997e+06,330,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,-6.187640e+08,-196307.711914,7.060131e+08
538,14,34,334280.425081,6.685609e+06,340,2.63258,0.345012,-0.361188,7.75,0.512913,0.71481,0.012288,-6.187640e+08,-196307.711914,7.060131e+08


In [372]:
random_poly(df, max_terms=4).describe()

count    540.000000
mean      -0.182192
std        0.045522
min       -0.250256
25%       -0.215294
50%       -0.171973
75%       -0.140937
max       -0.123204
Name: x_b, dtype: float64

In [286]:
t = lambda x: x + 1

In [111]:
import itertools

In [179]:
t = list(itertools.combinations(['x_b', 'QQ', 't', 'k'], np.random.randint(1, 5)))

In [180]:
t[0]


('x_b', 'QQ', 't', 'k')

In [181]:
list(t)[0]

('x_b', 'QQ', 't', 'k')

In [178]:
df.loc[:, t[0]]

,x_b,QQ,t,k
0,0.435095,1.74013,-0.380868,3.75
1,0.435095,1.74013,-0.380868,3.75
2,0.435095,1.74013,-0.380868,3.75
3,0.435095,1.74013,-0.380868,3.75
4,0.435095,1.74013,-0.380868,3.75
...,...,...,...,...
535,0.345012,2.63258,-0.361188,7.75
536,0.345012,2.63258,-0.361188,7.75
537,0.345012,2.63258,-0.361188,7.75
538,0.345012,2.63258,-0.361188,7.75
